# 深層学習

*Deep Learning*

深層学習を用いて言語モデルを作成する。  
ニューラルネットワークでマルコフモデルを作成し、深層学習における言語モデル実装の基礎を学ぶ。

In [1]:
import tensorflow_datasets as tfds
import MeCab
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import vocab, build_vocab_from_iterator

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
device

device(type='cpu')


---

## データセット

wiki40b

In [2]:
ds = tfds.load('wiki40b/ja', split='test')
ds = list(ds.as_numpy_iterator())

2023-06-19 01:10:53.332418: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-19 01:10:53.777461: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-19 01:10:54.361654: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-19 01:10:54.362394: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-19 01:10:54.362564: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_exec

In [3]:
data = []
for sample in ds:
    text = sample['text'].decode()
    sections = text.split('_START_SECTION_')
    for section in sections[1:]:
        sentence = section.split('_START_PARAGRAPH_')[1]
        sentence = sentence.replace('_NEWLINE_', '')
        sentence = sentence.replace('\n', '')
        data.append(sentence)

print('num of data:', len(data))
data[:5] # examples

num of data: 89698


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。',
 '香川県内の農業協同組合の信用事業を統括する県域農協系金融機関であり、県内農業協同組合を会員とする。香川県は全県単一農協の香川県農業協同組合となったが、先に単一農協となった奈良県や沖縄県のケースと異なり、信連の統合は行われなかった。通称は「JA香川信連」または「JAバンク香川」。統一金融機関コードは3037。主に法人顧客を中心としており、個人取引は殆どない。県内の大型商業施設にある、他金融機関管理の共同ATMには香川信連の管轄のものがある。',
 '534年（永熙3年）、独孤信の子として生まれた。独孤信が父母妻子を捨てて長安に入ったため、独孤羅は東魏に取り残されて高氏の虜囚となった。独孤

多すぎるので減らす

In [4]:
data = data[:100]


---

## 前処理

テキストをNNで扱える形に変換する。

### トークン化

文章をトークンごとに分割する。トークンとは文章を構成する最小の単位で、単語や句読点などが該当する。  
トークン化は分かち書きと似た意味であるが、言語モデルの領域で最小単位をトークンと呼ぶことが一般的なことや、単純にトークン化と呼ぶことが多いことから、本節以降トークン化と呼ぶ。

トークン化には、分かち書き同様形態素解析を用いる。各文章をトークンのリストとして格納する。

In [5]:
data_tokens = []
tagger = MeCab.Tagger('-Owakati')
for sentence in data:
    tokens = tagger.parse(sentence).strip().split(' ') # トークンのリスト
    data_tokens.append(tokens)

data_tokens[0][:10] # example

['「', '教科', '書', 'に', 'は', '決して', '載ら', 'ない', '」', '日本']

### ID化

NNでは文字列を扱えないので、トークン1つ1つにIDを割り当てる。

<br>

torchtextの`vocab`モジュールを使用する。

[torchtext.vocab — Torchtext 0.15.0 documentation](https://pytorch.org/text/stable/vocab.html#)

In [6]:
from torchtext.vocab import vocab, build_vocab_from_iterator

単語とその出現頻度を表した辞書を入力し、`Vocab`オブジェクトを作成する。`min_freq`で指定した出現頻度以下の単語は無視される。

In [7]:
v = vocab({'私': 4, 'りんご': 1, '食べる': 2, '好き': 3}, min_freq=1)
v(['私', 'りんご', '好き'])

[0, 1, 3]

`build_vocab_from_iterator()`でトークンのリスト（のリスト）から`vocab`オブジェクトを作成できる。

In [8]:
v = build_vocab_from_iterator(data)
v.get_itos()[:10] # vocabrary examples

['の', '、', 'に', 'ー', 'た', '。', 'る', 'と', 'は', 'を']

これを使ってトークン列をID列に変換する。

In [9]:
data_ids = []
v = build_vocab_from_iterator(data_tokens)
for tokens in data_tokens:
    data_ids.append(v(tokens))

print('num of vocabrary:', len(v))
data_ids[0][:10] # example

num of vocabrary: 4602


[17, 3681, 3742, 2, 4, 3873, 4415, 47, 19, 70]


---

## 学習データ

NNの学習を行うため、入力と出力のペアを作成する。  
今回はある単語から次に続く単語を予測するモデルを作成するので、ある単語IDとその次の単語IDがペアとなったデータを作成する。

In [10]:
class TextDataset(Dataset):
    def __init__(self, data_ids):
        x, y = [], []
        for ids in data_ids:
            for id1, id2 in zip(ids[:-1], ids[1:]):
                x.append(id1)
                y.append(id2)
        self.x = x
        self.y = y
        self.n_data = len(x)

    def __len__(self):
        return self.n_data

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

batch_size = 64
dataset = TextDataset(data_ids)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

n_data = len(dataset)
print('num of data:', n_data)

# examples
x, y = next(iter(dataloader))
print(x)
print(y)

num of data: 21128
tensor([ 628,   62,   30, 3761,    8, 3925, 1411,  113, 1336,  109,   11,    2,
          17,   55,   21,  388,    7,  266,    2,  390,    2,    9,    4,   28,
         469,    0,  142,    1, 1409,  990,  541,    0,  912,  956,  190,   97,
        3246,    3,    7,   22, 2547,   83, 1976, 1360,  736, 4242,    6,    5,
        4486, 2310,   38,    5,   10,    6,  748,   80, 1445,   27,  159,  385,
          55,    4,  249, 1206])
tensor([ 150,  306,   20,  291,    4,  914,  962,  212,  198,  384,    8, 1307,
         565, 3972,   27, 1567,   11,    4,  774,   59,    4,   24, 2784,    0,
          11, 1718,   12,  337, 1781, 1927,  187, 2269,    0,    7,  433,  252,
           2,   72,  190, 2689,   92,    0,   92,   21,   32, 1795,    3,   40,
        3740,    2,   26, 3218,  573,    7,   16,   10,    2,  426,  333,    5,
         385, 1742,   19,    0])



----

## モデル構築

マルコフモデルはある状態から次の状態を予測する。言語モデルに当てはめると、ある単語から次の単語を予測する。  
これをNNで実装するので、ある単語IDを入力に取り、次の単語IDを出力するモデルを作成する。

単語IDはカテゴリ変数なので、入力時はone-hotベクトルに変換する。出力は語彙数分の次元を持つベクトルを出力し、単語の分類問題として扱う。

In [11]:
class LanguageModel(nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.vocab_size = vocab_size
        self.net = nn.Sequential(
            nn.Linear(vocab_size, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, vocab_size)
        )

    def forward(self, x):
        x = F.one_hot(x, self.vocab_size).to(torch.float32)
        y = self.net(x)
        return y


---

## 学習

In [12]:
criterion = nn.CrossEntropyLoss()
def train(model, optimizer, n_epochs):
    model.train()
    for epoch in range(1, n_epochs + 1):
        loss_epoch = 0
        for x, t in dataloader:
            optimizer.zero_grad()
            x = x.to(device)
            t = t.to(device)
            y = model(x)
            loss = criterion(y, t)
            loss.backward()
            optimizer.step()
            loss_epoch += loss.item()
        print(f'{epoch}/{n_epochs} loss: {loss_epoch/len(dataloader)}')

In [13]:
model = LanguageModel(len(v), 512).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [14]:
train(model, optimizer, 10)

1/10 loss: 6.602865706037539
2/10 loss: 5.59666307527133
3/10 loss: 5.053603070019955
4/10 loss: 4.539914333568238
5/10 loss: 4.108413828823862
6/10 loss: 3.776285105961687
7/10 loss: 3.5165679447600486
8/10 loss: 3.3142312055628107
9/10 loss: 3.1581931164617627
10/10 loss: 3.0277433020830875
